### SQL Zillow Project
GitHub: @guzmanwolfrank

Volatile changes are occurring in the United States housing market.  Here we look at the data on Home Value to analyze which locations have the highest market values and which locations have sustained an above average move upward in price. 

In [ ]:
#Imports
import pandas as pd    
import numpy as np 
import seaborn as sns
import matplotlib as plt   
import warnings 

# Ignore all warnings 
warnings.filterwarnings('ignore')